<a href="https://colab.research.google.com/github/carmen-herlo/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/CarmenHernandez_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3 - AG3

Carmen Hernández

https://colab.research.google.com/drive/1G-Y5Mihf5MB_rLgKHPlfiAf6CxLNqhAD

https://github.com/carmen-herlo

# Problema del agente Viajero - Búsqueda aleatoria

In [0]:
#!pip install request

In [0]:
import urllib.request

file = 'swiss42.tsp'

urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp', file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f4dcec110b8>)

In [0]:
#!pip install tsplib95

In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)

nodos = list(problem.get_nodes())

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.wfunc(a,b)

#Se genera una solución aleatoria con comienzo en el nodo 0
def crear_solucion(nodos):
    solucion = [0]
    for _ in range(len(nodos)-1):
        solucion = solucion + [ random.choice( list(set(nodos)-set({0})-set(solucion)) )]
    return solucion


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
    distancia_ret = 0
    for i in range(len(solucion)-1):
        distancia_ret += distancia(solucion[i], solucion[i+1], problem)
    return distancia_ret + distancia(solucion[len(solucion)-1], solucion[0], problem)

In [0]:
# =============================================================================
# BÚSQUEDA ALEATORIA
# =============================================================================

def busqueda_aleatoria(problem, N):
    nodos = list(problem.get_nodes())
    
    mejor_solucion = []
    mejor_distancia = 10e10
    
    for i in range(N):
        solucion = crear_solucion(nodos)
        distancia_solucion = distancia_total(solucion, problem)
        
        if distancia_solucion < mejor_distancia:
            mejor_solucion = solucion
            mejor_distancia = distancia_solucion
            
    print('Mejor distancia: ',mejor_distancia)
    print('Mejor solución: ',mejor_solucion)
    
    return mejor_solucion

mejor_solucion = busqueda_aleatoria(problem, 100)

Mejor distancia:  4042
Mejor solución:  [0, 40, 9, 28, 31, 25, 7, 4, 41, 11, 26, 13, 10, 21, 23, 22, 39, 30, 24, 20, 36, 27, 33, 37, 16, 35, 15, 19, 18, 5, 34, 1, 2, 17, 14, 12, 38, 6, 3, 8, 32, 29]


# Problema del agente Viajero - Búsqueda local

In [0]:
def generar_vecina(solucion, problem):
    mejor_solucion = []
    mejor_distancia = 10e10
    for i in range(1, len(solucion)-1): #desde el segundo nodo hasta el penúltimo
        for j in range(i+1, len(solucion)):
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] #intercambio de i por j
            distancia_vecina = distancia_total(vecina, problem)
            if distancia_vecina <= mejor_distancia:
                mejor_solucion = vecina
                mejor_distancia = distancia_vecina
    return mejor_solucion

In [0]:
# =============================================================================
# BÚSQUEDA LOCAL
# =============================================================================

def busqueda_local(problem, N):
    mejor_solucion = []
    mejor_distancia = 10e10
    
    nodos = list(problem.get_nodes())
    
    solucion_referencia = crear_solucion(nodos)
    
    for i in range (N):
        vecina = generar_vecina(solucion_referencia, problem)
        distancia_vecina = distancia_total(vecina, problem)
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        solucion_referencia = mejor_solucion
        
    print('Mejor distancia: ',distancia_total(mejor_solucion, problem))
    print('Mejor solución: ',mejor_solucion)
    
        
    return mejor_solucion

mejor_solucion = busqueda_local(problem, 100)

Mejor distancia:  1767
Mejor solución:  [0, 31, 35, 36, 17, 37, 7, 1, 3, 27, 2, 4, 18, 12, 11, 25, 10, 34, 33, 20, 32, 30, 29, 38, 22, 39, 21, 24, 40, 41, 26, 5, 13, 19, 14, 16, 15, 6, 8, 23, 9, 28]


# Problema del agente Viajero - Recocido simulado

In [0]:
def probabilidad(T, d):
    r = random.random()
    if r < e**(-1*d/T):
#    if r >= e**(-1*(d/(T**.5*10**(-5)))):
        return True
    else:
        return False

def bajar_temperatura(T):
    return T*0.999

In [0]:
# =============================================================================
# RECOCIDO SIMULADO
# =============================================================================

def recocido_simulado(problem, TEMPERATURA):
    
    solucion_referencia = crear_solucion(nodos)
    
    distancia_referencia = distancia_total(solucion_referencia, problem)
    
    mejor_solucion = []
    mejor_distancia = 10e10
    
    while TEMPERATURA > 1: 
        vecina = generar_vecina(solucion_referencia, problem)
        distancia_vecina = distancia_total(vecina, problem)
        
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia-distancia_vecina)):
            solucion_referencia = vecina
            distancia_referencia = distancia_vecina
            
        TEMPERATURA = bajar_temperatura(TEMPERATURA)
        
        print('Mejor distancia: ',mejor_distancia)
        print('Mejor solución: ',mejor_solucion)
        
        return mejor_solucion
        
mejor_solucion = recocido_simulado(problem, 1000)

Mejor distancia:  4975
Mejor solución:  [0, 20, 38, 23, 14, 4, 40, 32, 6, 2, 31, 29, 21, 36, 37, 12, 35, 18, 1, 22, 30, 7, 24, 39, 8, 27, 17, 3, 11, 34, 26, 15, 9, 33, 25, 16, 19, 13, 28, 10, 5, 41]


# Problema del agente Viajero - Colonia de Hormigas

In [0]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================
#Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.wfunc(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
    distancia_ret = 0
    for i in range(len(solucion)-1):
        distancia_ret += distancia(solucion[i], solucion[i+1], problem)
    return distancia_ret + distancia(solucion[len(solucion)-1], solucion[0], problem)

def Add_Nodo(problem, H, T):
    #Mejora: Establecer una función de probabilidad para añadir
    #un nuevo nodo dependiendo de los nodos más cercanos y de las feromonas depositadas
    Nodos = list(problem.get_nodes())
    return random.choice( list (set(range(1,len(Nodos))) - set(H)) )

def Incrementa_Feromona(problem, T, H):
    #Incrementa según la calidad de la solución. Añadir una cantidad inversamente 
    #proporcional a la distancia total
    for i in range(len(H)-1):
        T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
    return T

def Evaporar_Feromona(T, Nodos):
    #Evapora 0.3 el valor de la feromona, sin que baje de 1
    #Mejora: Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad
    #actual y de la suma total de feromonas depositadas...
    T = [[ max(T[i][j] - 0.3, 1) for i in range(len(Nodos))] for j in range(len(Nodos))]
    return T

In [0]:
# =============================================================================
# ESQUEMA BÁSICO - 1º ITERACIÓN
# =============================================================================
def hormigas(problem, N):
    #problem = datos del problema
    #N = Número de agentes (hormigas)
    
    #Nodos
    Nodos = list(problem.get_nodes())
    #Aristas
    Aristas = list(problem.get_edges())
    
    #Inicializa las aristas con una cantidad inicial de feromonas:1
    #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
    T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
    
    #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0 
    Hormiga = [[0] for _ in range(N)]
    
    #Recorre cada agente construyendo la solución
    for h in range(N):
        #Para cada agente se construye un camino
        for i in range(len(Nodos)-1):
            
            #Elige el siguiente nodo
            Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T)
            Hormiga[h].append(Nuevo_Nodo)
    
        #Incremente feromonas en esa arista
        T = Incrementa_Feromona(problem, T, Hormiga[h])
        #print("Feromonas(1)",T)
        
        #Evapora feromonas
        T = Evaporar_Feromona(T, Nodos)
        #print("Feromonas(2)",T)
        
    #Seleccionamos el mejor agente
    mejor_solucion = []
    mejor_distancia = 10e100
    for h in range(N):
        distancia_actual = distancia_total(Hormiga[h], problem)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[h]
            mejor_distancia = distancia_actual
    
    print('Mejor distancia: ',mejor_distancia)
    print('Mejor solución: ',mejor_solucion)
        
    return mejor_solucion
    
    

mejor_solucion = hormigas(problem,100)

Mejor distancia:  4021
Mejor solución:  [0, 30, 13, 29, 1, 12, 33, 37, 17, 10, 6, 36, 16, 15, 31, 3, 20, 32, 11, 4, 22, 28, 40, 25, 8, 41, 9, 24, 21, 23, 26, 19, 7, 34, 14, 39, 38, 18, 27, 2, 35, 5]
